In [90]:
import cv2
import numpy as np
import random
import sys 
import re
# from matplotlib import pyplot as plt

# !pip install pillow
# !pip install pytesseract
# !pip install opencv-python

from PIL import Image
import pytesseract
import argparse
# import cv2
import os

In [2]:
def showImg(img, title = str(round(random.random()*1000,0))):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyWindow(title)

In [3]:
def cropImage(image, offsetX, offsetY, width, height):
    cardNu_threshold = 10
    minX = offsetX
    minY = offsetY
    cardNuWidth = width
    cardNuHeight = height

    return image[minY - cardNu_threshold:minY + cardNuHeight + cardNu_threshold, minX -
                   cardNu_threshold:minX + cardNuWidth + cardNu_threshold]


## Util functions for extract text

In [72]:
# remove background, just keep text color from image
# imput image instance
def keepColor(image, lower, upper):
    frame = image.copy()
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # cv.imshow("hsv", hsv)
    # define range of blue color in HSV

    # Threshold the HSV image to get only blue colors
    res = cv2.inRange(hsv, lower, upper)
    # Bitwise-AND mask and original image
#     res = cv2.bitwise_and(frame,frame, mask= mask)
#     cv2.imshow('frame',frame)
#     cv2.imshow('mask',mask)
#     cv2.imshow('res',res)
#     cv2.waitKey(0) 
#     cv2.destroyAllWindows()
    
    
    return res
    

In [73]:
def extractText(image):
    
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

    # Write down temporary file to extract
    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, image)

    # extract text from image using tesseract ocr
    text = pytesseract.image_to_string(Image.open(filename))

    # Remove temporary image
    os.remove(filename)

    return text

## 1. Get Id number function

In [121]:
# get Id number function
# key color = in range [30 - 80]
def getIdNumber(imagePath, V_upper_color=50):
    #load image
    originImg = cv2.imread(imagePath)
#     showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_img = cropImage(originImg, 400, 140, 350, 40)
#     showImg(cropped_img, "Cropped")

    #filter
    
    lower = np.array([0,0,0])
    upper = np.array([180, 255, V_upper_color]) # V in range (50 - 80) is fine

    filteredImg = keepColor(cropped_img, lower, upper)
#     showImg(filteredImg)
    
    #extract text
    res = extractText(filteredImg)
    
    #remove non digit characters
    res = re.sub("\\D", '', res)
    return res


In [123]:
print(getIdNumber('./data/temp4.jpg', 70)) #key

225682227
